In [1]:
import numpy as np
import xarray as xr
import os
import pandas as pd
import glob

In [3]:
### import some analysis functions we wrote for this project
import sys ; sys.path.append("..")
from ppe_tools import Ensemble,Member,ParamInfo
from ppe_tools.utils import *

In [29]:
#this is the base paramfile that I will copy and edit
#important that the paramfile matches the version of CTSM
basefile = '/glade/p/cgd/tss/people/oleson/modify_param/ctsm51_params.c210507_kwo.c220322.nc'

#where to write the new files:
paramdir = '/glade/scratch/djk2120/PPEn11/paramfiles/'
nldir    = '/glade/scratch/djk2120/PPEn11/namelist_mods/'
prefix   = 'OAAT'

#make the directories, if needed 
for d in [paramdir,nldir]:
    if ~os.path.isdir(d):
        os.system('mkdir -p '+d)
        
#diagnose next available paramfile in sequence      
lastfile = sorted(glob.glob(paramdir+prefix+'*.nc'))[-1]
nextnum  = 1+int(lastfile.split(prefix)[1].split('.nc')[0])

In [6]:
#read in the spreadsheet
# make sure to first publish the spreadsheet
###  file > share > publish to web > csv
csv = 'phen_apr2022.csv'
if not os.path.isfile(csv):
    #data_url needs to be '"foo.bar"'
    data_url = '"https://docs.google.com/spreadsheets/d/e/2PACX-1vQbnPXdKgC4HjYddGl7qOUM6jxTqgqin2uoU2YU5wvbrpbWRuF53PU0k-Dz_Hg445v9yUsROSFEwYDq/pub?gid=719055511&single=true&output=csv"'
    cmd = 'curl -L '+data_url+' > '+csv
    os.system(cmd)

In [90]:
#parse the the csv into an oaats dictionary
included = data['test']=='N'
included[165]=False #remove duplicate
params,oaats=parse_csv(csv,included=included)

In [91]:
params

,name,location,min,max,pft_mins,pft_maxs,flag
0,dsladlai,P,20percent,20percent,NaN,NaN,NaN
1,a_fix,P,-4.14,-3.1,NaN,NaN,NaN
2,b_fix,P,0.23,0.31,NaN,NaN,NaN
3,c_fix,P,24.49,25.81,NaN,NaN,NaN
4,s_fix,P,20percent,20percent,NaN,NaN,NaN
5,crit_onset_gdd_sf,P,20percent,20percent,NaN,NaN,NaN
6,crit_dayl,P,20percent,20percent,NaN,NaN,NaN
7,crit_offset_fdd,P,20percent,20percent,NaN,NaN,NaN
8,crit_offset_swi,P,20percent,20percent,NaN,NaN,NaN
9,crit_onset_swi,P,20percent,20percent,NaN,NaN,NaN


In [92]:
oaats

{'dsladlai': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'a_fix': {'loc': 'P', 'min': array(-4.14), 'max': array(-3.1)},
 'b_fix': {'loc': 'P', 'min': array(0.23), 'max': array(0.31)},
 'c_fix': {'loc': 'P', 'min': array(24.49), 'max': array(25.81)},
 's_fix': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'crit_onset_gdd_sf': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'crit_dayl': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'crit_offset_fdd': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'crit_offset_swi': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'crit_onset_swi': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'soilpsi_off': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'soilpsi_on': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'fstor2tran': {'loc': 'P', 'min': '20percent', 'max': '20percent'},
 'bdnr': {'loc': 'P', 'min': '20percent', 'max': '20percent'}}

In [96]:
x    = Ensemble(basefile,paramdir,nldir)
x.add_oaats(oaats,prefix,nextnum,skipBFB=True)
x.write(oaatfile='/glade/scratch/djk2120/PPEn11/phenology.csv')